<a href="https://colab.research.google.com/github/amidavidbhai/VerificationQC/blob/main/QuantumVerification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --quiet qutip qutip-qip&> /dev/null

In [2]:
import qutip as qt
import numpy as np
import warnings
from scipy.linalg import polar

warnings.filterwarnings('ignore', category=FutureWarning)

In [3]:
# ============================================================
# Global configuration: gate + protocol parameters
# ============================================================

# Biased coin: V(λ) = R_y(2λ), here λ = π/4 → θ_gate = π/2
LAMBDA_COIN = np.pi / 4.0
THETA_GATE = 2.0 * LAMBDA_COIN
IDEAL_COIN_P1 = np.sin(LAMBDA_COIN) ** 2  # should be 0.5

# XOR classifier parameters are implicit in U_xor(x1, x2)

# Verification thresholds
EPS_D_PASS   = 0.02   # process infidelity threshold for PASS
EPS_LOG_PASS = 0.05   # coherent deviation threshold for PASS
EPS_INC_PASS = 0.05   # noise overhead threshold for PASS

D_HARD       = 0.30   # above this, consider hard fail
LOG_HARD     = 0.90   # ~almost orthogonal
INC_HARD     = 0.50   # large noise

# ============================================================
# Regimes: defining all cases here
# ============================================================

REGIMES = [
    {
        "name": "Calibrated",
        "params": {
            "J1Q": 0.0, "JQ2": 0.0,
            "A1": 0.0, "A2": 0.0, "Aro": 0.0,
            "delta1": 0.0, "delta2": 0.0, "delta_ro": 0.0
        }
    },
    {
        "name": "Moderate",
        "params": {
            "J1Q": 0.5, "JQ2": 0.5,         # high static coupling
            "A1": 0.0, "A2": 0.0, "Aro": 0.0,  # no drives
            "delta1": 10.0, "delta2": 10.0, "delta_ro": 0.0
        }
    },
    {
        "name": "Worst-case",
        "params": {
            "J1Q": 0.5, "JQ2": 0.5,           # high static coupling
            "A1": 0.5, "A2": 0.5, "Aro": 0.5, # max neighbor & readout drives
            "delta1": 10.0, "delta2": 10.0, "delta_ro": 0.0
        }
    }
]

In [4]:
# ============================================================
# Basic single-qubit unitaries: Rx, Ry, Rz
# ============================================================

def Rx(theta):
    return qt.Qobj([[np.cos(theta/2), -1j*np.sin(theta/2)],
                    [-1j*np.sin(theta/2), np.cos(theta/2)]])

def Ry(theta):
    return qt.Qobj([[np.cos(theta/2), -np.sin(theta/2)],
                    [np.sin(theta/2),  np.cos(theta/2)]])

def Rz(theta):
    return qt.Qobj([[np.exp(-1j*theta/2), 0.0],
                    [0.0, np.exp(1j*theta/2)]])

In [5]:
# ============================================================
# Pulse shape and Hamiltonian simulation
# ============================================================

def pulse_shape(shape="cos", A=1.0, delta=0.0, sigma=0.1,
                alpha=0.5, chirp_rate=0.0, t_duration=1.0):
    t_center = t_duration / 2
    if shape == "cos":
        return lambda t, args: A * np.cos(delta * t)
    elif shape == "gaussian":
        return lambda t, args: A * np.exp(-((t - t_center)**2) / (2 * sigma**2))
    elif shape == "square":
        return lambda t, args: A if (t_center - 0.2*t_duration) <= t <= (t_center + 0.2*t_duration) else 0.0
    elif shape == "chirp":
        return lambda t, args: A * np.cos((delta + chirp_rate * t) * t)
    elif shape == "drag":
        return lambda t, args: A * (
            np.exp(-((t - t_center)**2)/(2 * sigma**2))
            - alpha * (t - t_center) / sigma**2 * np.exp(-((t - t_center)**2)/(2 * sigma**2))
        )
    else:
        raise ValueError(f"Unknown pulse shape: {shape}")

def get_final_state_for_qpt(qubit_Q_input_dm, theta_gate, rest_params):
    """
    Simulate H(t) = H_Q(t) + H_rest(t) for a specific 1-qubit input DM on Q.
    Neighbors q1 and q2 start in |0>.
    """
    I = qt.qeye(2)
    zero = qt.basis(2, 0)

    # Initial 3-qubit state: |0>_q1 ⊗ rho_Q ⊗ |0>_q2
    state_init_3q = qt.tensor(zero.proj(), qubit_Q_input_dm, zero.proj())

    t_duration = 1.0
    tlist = np.linspace(0, t_duration, 50)

    # 1. H_rest(t)
    H_couple_1Q = rest_params['J1Q'] * qt.tensor(qt.sigmay(), qt.sigmax(), I)
    H_couple_Q2 = rest_params['JQ2'] * qt.tensor(I, qt.sigmax(), qt.sigmay())
    H_static = H_couple_1Q + H_couple_Q2

    coeff1 = pulse_shape(shape="chirp",
                         A=rest_params['A1'],
                         delta=rest_params['delta1'],
                         chirp_rate=0.5,
                         t_duration=t_duration)
    coeff2 = pulse_shape(shape="chirp",
                         A=rest_params['A2'],
                         delta=rest_params['delta2'],
                         chirp_rate=0.5,
                         t_duration=t_duration)
    H_drive1 = [qt.tensor(qt.sigmax(), I, I), coeff1]
    H_drive2 = [qt.tensor(I, I, qt.sigmax()), coeff2]

    coeff_ro = pulse_shape(shape="cos",
                           A=rest_params['Aro'],
                           delta=rest_params['delta_ro'],
                           t_duration=t_duration)
    H_ro = [qt.tensor(I, qt.sigmax(), I), coeff_ro]

    # 2. H_Q(t): Gaussian drive on Q in Y
    sigma_gate = 0.2
    A_Q = (theta_gate / 2.0) / (sigma_gate * np.sqrt(2 * np.pi))
    coeff_Q = pulse_shape(shape="gaussian",
                          A=A_Q, sigma=sigma_gate,
                          t_duration=t_duration)
    H_gate_Q = [qt.tensor(I, qt.sigmay(), I), coeff_Q]

    # 3. Combine and simulate
    H_total = [H_static, H_drive1, H_drive2, H_ro, H_gate_Q]
    sol = qt.mesolve(H_total, state_init_3q, tlist, c_ops=[], e_ops=[])

    rho_final_total = sol.states[-1]
    rho_Q_final = rho_final_total.ptrace(1)  # trace out q1, q2
    return rho_Q_final

In [6]:
# ============================================================
# QPT and Choi reconstruction
# ============================================================

def perform_qpt_pipeline(theta_gate, rest_params):
    """
    Full single-qubit QPT on Q to reconstruct C_choi for E_phys.
    Returns C_choi as 4x4 Qobj.
    """
    psi_0 = qt.basis(2, 0)
    psi_1 = qt.basis(2, 1)
    psi_plus = (psi_0 + psi_1).unit()
    psi_plus_i = (psi_0 + 1j * psi_1).unit()

    rho_in_list = [
        psi_0.proj(),      # |0><0|
        psi_1.proj(),      # |1><1|
        psi_plus.proj(),   # |+><+|
        psi_plus_i.proj()  # |+i><+i|
    ]

    rho_out_list = []
    for rho_in in rho_in_list:
        rho_out = get_final_state_for_qpt(rho_in, theta_gate, rest_params)
        rho_out_list.append(rho_out)

    rho0_out, rho1_out, rho_plus_out, rho_plus_i_out = rho_out_list

    A_out = rho0_out
    B_out = rho1_out

    S = A_out + B_out
    X = 2 * rho_plus_out - S
    Y = 2 * rho_plus_i_out - S

    C_out = 0.5 * (X + 1j * Y)  # E(|0><1|)
    D_out = 0.5 * (X - 1j * Y)  # E(|1><0|)

    zero = qt.basis(2, 0)
    one  = qt.basis(2, 1)

    proj00 = zero * zero.dag()
    proj11 = one * one.dag()
    proj01 = zero * one.dag()
    proj10 = one * zero.dag()

    C_choi = (
        qt.tensor(proj00, A_out) +
        qt.tensor(proj01, C_out) +
        qt.tensor(proj10, D_out) +
        qt.tensor(proj11, B_out)
    )
    return C_choi

def kraus_operators_from_choi(C_choi, tol=1e-8):
    """
    Extract Kraus operators from a Choi matrix C (4x4 Qobj).
    Includes a normalization step to enforce trace preservation.
    """
    C_mat = C_choi.full()
    eigvals, eigvecs = np.linalg.eigh(C_mat)

    d = 2
    kraus_ops_raw = []
    for k in range(len(eigvals)):
        lam = eigvals[k]
        if np.real(lam) > tol:
            v = eigvecs[:, k]
            K_mat = np.sqrt(lam) * v.reshape((d, d), order='F')
            kraus_ops_raw.append(qt.Qobj(K_mat))

    if not kraus_ops_raw:
        return [qt.Qobj(np.zeros((2, 2), dtype=complex))]

    sum_ops = sum([K.dag() * K for K in kraus_ops_raw])
    trace_val = np.real(sum_ops.tr()) / d
    if trace_val <= 0:
        norm_factor = 1.0
    else:
        norm_factor = 1.0 / np.sqrt(trace_val)

    kraus_ops_normalized = [K * norm_factor for K in kraus_ops_raw]
    return kraus_ops_normalized

def choi_from_kraus(kraus_ops):
    """
    Given Kraus operators {K_i} for a 1-qubit channel, build its Choi matrix:
        C = sum_i vec(K_i) vec(K_i)^\dagger
    using column-stacking (Fortran order) vectorization.
    """
    d = kraus_ops[0].shape[0]  # d = 2
    C = np.zeros((d*d, d*d), dtype=complex)
    for K in kraus_ops:
        Kmat = K.full()
        v = Kmat.reshape(d*d, order='F')  # column vectorization
        C += np.outer(v, v.conj())
    return qt.Qobj(C)

<>:88: SyntaxWarning: invalid escape sequence '\d'
<>:88: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-1407188518.py:88: SyntaxWarning: invalid escape sequence '\d'
  C = sum_i vec(K_i) vec(K_i)^\dagger


In [7]:
# ============================================================
# Isometry fit: E_phys ≈ N ∘ U_eff
# ============================================================

def su2_axis_angle(U_qobj, tol=1e-10):
    """
    Given a 2x2 unitary Qobj, return (theta, n_axis, U_su2)
    with det(U_su2) = 1 and U_su2 = exp(-i * theta/2 * n·σ).
    """
    U = U_qobj.full()
    detU = np.linalg.det(U)
    phi = np.angle(detU) / 2.0
    U_su2 = U * np.exp(-1j * phi)

    trU = np.trace(U_su2)
    cos_half_theta = np.real(trU) / 2.0
    cos_half_theta = np.clip(cos_half_theta, -1.0, 1.0)
    theta = 2.0 * np.arccos(cos_half_theta)

    if abs(theta) < tol:
        return 0.0, np.array([0.0, 0.0, 1.0]), qt.Qobj(U_su2)

    sigmas = [qt.sigmax(), qt.sigmay(), qt.sigmaz()]
    n = []
    for sigma in sigmas:
        val = np.trace(sigma.full().conj().T @ U_su2)
        n_k = (1j * val) / (2.0 * np.sin(theta / 2.0))
        n.append(np.real(n_k))

    n = np.array(n)
    n_norm = np.linalg.norm(n)
    if n_norm > tol:
        n = n / n_norm
    else:
        n = np.array([0.0, 0.0, 1.0])

    return float(theta), n, qt.Qobj(U_su2)

def fit_unitary_plus_noise_from_choi(C_choi, tol=1e-8):
    """
    Given a Choi matrix C_choi (4x4 Qobj) for a 1-qubit channel E_phys,
    factor E_phys ≈ N ∘ U_eff where:

      - U_eff is a 1-qubit unitary (dominant coherent part)
      - N is a Pauli-like noise channel with Kraus operators L_i.
    """
    # 1. Get TP Kraus operators K_i of E_phys
    kraus_ops = kraus_operators_from_choi(C_choi, tol=tol)

    # 2. Pick dominant Kraus by Frobenius norm
    fro_norms = [np.linalg.norm(K.full(), 'fro') for K in kraus_ops]
    k0_index = int(np.argmax(fro_norms))
    K0 = kraus_ops[k0_index]

    # Optional: singular values of K0 to check "mostly unitary"
    svals = np.linalg.svd(K0.full(), compute_uv=False)
    dominant_sv = float(np.max(svals))

    # 3. Polar decomposition: K0 = U_eff · H  (U_eff unitary)
    from scipy.linalg import polar
    U_eff_mat, H = polar(K0.full())
    U_eff = qt.Qobj(U_eff_mat)

    # 4. Extract axis-angle for U_eff (for reporting)
    theta_eff, n_axis, U_su2 = su2_axis_angle(U_eff)

    # 5. Define noise channel N via L_i = K_i · U_eff.dag()
    L_ops = [K * U_eff.dag() for K in kraus_ops]

    # 6. Compute Pauli-transfer diagonal for N (approximate Pauli channel)
    paulis = [qt.qeye(2), qt.sigmax(), qt.sigmay(), qt.sigmaz()]
    lambdas = []

    for j in range(1, 4):  # X, Y, Z
        sigma_j = paulis[j]
        N_sigma_j = sum([L * sigma_j * L.dag() for L in L_ops])
        lam_j = 0.5 * np.trace(sigma_j.full().conj().T @ N_sigma_j.full()).real
        lambdas.append(lam_j)

    lam_x, lam_y, lam_z = lambdas

    # 7. Map lambdas to Pauli probabilities (Pauli channel approximation)
    p_I = (1 + lam_x + lam_y + lam_z) / 4.0
    p_X = (1 + lam_x - lam_y - lam_z) / 4.0
    p_Y = (1 - lam_x + lam_y - lam_z) / 4.0
    p_Z = (1 - lam_x - lam_y + lam_z) / 4.0

    ps = np.array([p_I, p_X, p_Y, p_Z])
    ps = np.clip(ps, 0.0, 1.0)
    if ps.sum() > 0:
        ps = ps / ps.sum()

    sum_L = sum([L.dag() * L for L in L_ops])

    return {
        "U_eff": U_eff,
        "theta_eff": float(theta_eff),
        "n_axis": n_axis,
        "dominant_singular_value": dominant_sv,
        "pauli_lambdas": (lam_x, lam_y, lam_z),
        "pauli_probs": ps,
        "kraus_noise": L_ops,
        "K0_index": k0_index,
        "sum_L_dag_L": sum_L
    }

In [8]:
# ============================================================
# Ideal channel for Ry(θ) and verification metrics
# ============================================================

def ideal_choi_ry(theta):
    U = Ry(theta)

    zero = qt.basis(2, 0)
    one  = qt.basis(2, 1)
    proj00 = zero * zero.dag()
    proj11 = one * one.dag()
    proj01 = zero * one.dag()
    proj10 = one * zero.dag()

    def E(rho):
        return U * rho * U.dag()

    A_out = E(proj00)
    B_out = E(proj11)
    C_out = E(proj01)
    D_out = E(proj10)

    C_choi = (
        qt.tensor(proj00, A_out) +
        qt.tensor(proj01, C_out) +
        qt.tensor(proj10, D_out) +
        qt.tensor(proj11, B_out)
    )
    return C_choi

def process_distance_D(C_ideal, C_phys):
    d = 2
    C_i = C_ideal.full()
    C_p = C_phys.full()
    F_proc = (1.0 / d**2) * np.trace(C_i.conj().T @ C_p)
    return float(1.0 - np.real(F_proc))

def delta_log(U_ideal, U_eff):
    M = U_ideal.dag() * U_eff
    val = M.tr()
    return float(np.sqrt(max(0.0, 1.0 - (abs(val)**2) / 4.0)))

def delta_inc(pauli_probs):
    p_I = pauli_probs[0]
    return float(1.0 - p_I)

In [9]:
# ============================================================
# Applying a channel from its Choi matrix
# ============================================================

def apply_channel_from_choi(C_choi, rho_in):
    """
    Apply the 1-qubit channel whose Choi matrix was built as in perform_qpt_pipeline:
        C = |0><0| ⊗ A_out + |0><1| ⊗ C_out
          + |1><0| ⊗ D_out + |1><1| ⊗ B_out.
    Then for ρ = [[ρ00, ρ01], [ρ10, ρ11]]:
        E(ρ) = ρ00 A_out + ρ01 C_out + ρ10 D_out + ρ11 B_out.
    """
    M = C_choi.full()

    # Block structure: [[A_out, C_out],
    #                   [D_out, B_out]]
    A_out = qt.Qobj(M[0:2, 0:2])
    C_out = qt.Qobj(M[0:2, 2:4])
    D_out = qt.Qobj(M[2:4, 0:2])
    B_out = qt.Qobj(M[2:4, 2:4])

    rm = rho_in.full()
    r00 = rm[0, 0]
    r01 = rm[0, 1]
    r10 = rm[1, 0]
    r11 = rm[1, 1]

    rho_out = (
        r00 * A_out +
        r01 * C_out +
        r10 * D_out +
        r11 * B_out
    )
    return rho_out

In [10]:
# ============================================================
# Protocol-level metrics: coin and XOR
# ============================================================

def biased_coin_prob_from_channel(C_choi, lam):
    """
    Given a *noise* channel C_choi and a bias λ, compute
    P(|1>) after ideal biased-coin state + noise.
    Ideal coin state: |ψ> = cos λ |0> + sin λ |1>.
    """
    psi0 = qt.basis(2, 0)
    psi1 = qt.basis(2, 1)

    psi_ideal = np.cos(lam) * psi0 + np.sin(lam) * psi1
    rho_ideal = psi_ideal * psi_ideal.dag()

    rho_out = apply_channel_from_choi(C_choi, rho_ideal)
    proj1 = psi1 * psi1.dag()
    return float((proj1 * rho_out).tr().real)


def coin_p1_from_channel(C_choi):
    zero = qt.basis(2, 0)
    one = qt.basis(2, 1)
    proj0 = zero * zero.dag()
    proj1 = one * one.dag()

    rho_out = apply_channel_from_choi(C_choi, proj0)
    p1 = (proj1 * rho_out).tr().real
    return float(p1)

def U_xor(x1, x2):
    """
    U_XOR(x1,x2) = Rx((2x2 - 1)π/2) Rz((2x1 - 1)π/2) H
    """
    ang_z = (2 * x1 - 1) * np.pi / 2.0
    ang_x = (2 * x2 - 1) * np.pi / 2.0
    H = qt.Qobj([[1, 1], [1, -1]]) / np.sqrt(2)
    return Rx(ang_x) * Rz(ang_z) * H

def xor_accuracy_from_channel(C_choi):
    zero = qt.basis(2, 0)
    proj0 = zero * zero.dag()

    accs = []
    for x1 in [0, 1]:
        for x2 in [0, 1]:
            U = U_xor(x1, x2)
            rho_in = proj0
            rho_after_U = U * rho_in * U.dag()
            rho_after_noise = apply_channel_from_choi(C_choi, rho_after_U)

            y = x1 ^ x2
            proj_y = qt.basis(2, y) * qt.basis(2, y).dag()
            p_correct = (proj_y * rho_after_noise).tr().real
            accs.append(p_correct)

    return float(np.mean(accs))

In [11]:
# ============================================================
# Verdict classification
# ============================================================

def classify_verdict(D_val, dlog, dinc):
    if (D_val <= EPS_D_PASS and
        dlog <= EPS_LOG_PASS and
        dinc <= EPS_INC_PASS):
        return "PASS"
    if (D_val >= D_HARD or
        dlog >= LOG_HARD or
        dinc >= INC_HARD):
        return "HARD FAIL"
    return "SOFT FAIL"

# ============================================================
# Main analysis for each regime
# ============================================================

def analyze_regime(regime_name, rest_params, theta_gate):
    # 1. Channel reconstruction: full physical channel E_phys
    C_phys = perform_qpt_pipeline(theta_gate, rest_params)
    C_ideal = ideal_choi_ry(theta_gate)

    # 2. Channel-level metrics from E_phys
    D_val  = process_distance_D(C_ideal, C_phys)
    fit_res = fit_unitary_plus_noise_from_choi(C_phys)

    U_eff       = fit_res["U_eff"]
    pauli_probs = fit_res["pauli_probs"]
    L_ops       = fit_res["kraus_noise"]   # Kraus operators for N

    U_ideal = Ry(theta_gate)
    dlog = delta_log(U_ideal, U_eff)
    dinc = delta_inc(pauli_probs)

    verdict = classify_verdict(D_val, dlog, dinc)

    # 3. Build Choi for the **noise channel** N from {L_i}
    C_noise = choi_from_kraus(L_ops)

    # 4. Protocol-level metrics: apply N after ideal circuits
    coin_p = biased_coin_prob_from_channel(C_noise, LAMBDA_COIN)
    xor_acc = xor_accuracy_from_channel(C_noise)

    return {
        "name": regime_name,
        "params": rest_params,
        "D": D_val,
        "delta_log": dlog,
        "delta_inc": dinc,
        "verdict": verdict,
        "coin_p1": coin_p,
        "xor_acc": xor_acc
    }

In [12]:
# ============================================================
# Running all regimes and printing a compact summary
# ============================================================

if __name__ == "__main__":
    header = (
        "Regime, "
        "J1Q, JQ2, A1, A2, Aro, delta1, delta2, delta_ro, "
        "D, δ_log, δ_inc, "
        "P_coin(|1>), XOR_acc, Verdict"
    )
    print(header)

    for reg in REGIMES:
        name = reg["name"]
        params = reg["params"]
        res = analyze_regime(name, params, THETA_GATE)

        p = res["params"]
        line = (
            f"{res['name']}, "
            f"{p['J1Q']:.3f}, {p['JQ2']:.3f}, "
            f"{p['A1']:.3f}, {p['A2']:.3f}, {p['Aro']:.3f}, "
            f"{p['delta1']:.3f}, {p['delta2']:.3f}, {p['delta_ro']:.3f}, "
            f"{res['D']:.4f}, {res['delta_log']:.4f}, {res['delta_inc']:.4f}, "
            f"{res['coin_p1']:.4f}, {res['xor_acc']:.4f}, {res['verdict']}"
        )
        print(line)

Regime, J1Q, JQ2, A1, A2, Aro, delta1, delta2, delta_ro, D, δ_log, δ_inc, P_coin(|1>), XOR_acc, Verdict
Calibrated, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.0001, 0.0098, 0.0000, 0.5000, 1.0000, PASS
Moderate, 0.500, 0.500, 0.000, 0.000, 0.000, 10.000, 10.000, 0.000, 0.2687, 0.1065, 0.2605, 0.6174, 0.8850, SOFT FAIL
Worst-case, 0.500, 0.500, 0.500, 0.500, 0.500, 10.000, 10.000, 0.000, 0.3377, 0.3907, 0.2618, 0.6059, 0.8569, HARD FAIL
